In [4]:
import glob

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [6]:
args = pickle.load(open('arguments.pickle', 'rb'))

In [7]:
from Bio import SeqIO

def batch_iterator(iterator, batch_size):
    entry = True  # Make sure we loop once

    while entry:
        batch = []

        while len(batch) < batch_size:
            try:
                entry = next(iterator)
            except StopIteration:
                entry = False

            if not entry:
                # End of file
                break

            batch.append(entry)

        if batch:
            yield batch


In [25]:
align_dir = os.path.join(args.genome_dir, 'align')
if not os.path.exists(align_dir):
    os.mkdir(align_dir)

data_list = []
n=0
for name in glob.glob(os.path.join(args.genome_dir, '*.fasta')):
    
    record_iter = SeqIO.parse(name, 'fasta')
    for i, batch in enumerate(batch_iterator(record_iter, 500), start=1):
        filename = os.path.join(align_dir, f'group{n}_{i}.fasta')
        count = SeqIO.write(batch, filename, 'fasta')
        print('Wrote {} records to {}'.format(count, filename))
    n+=1

Wrote 500 records to genome_dir/align/group0_1.fasta
Wrote 500 records to genome_dir/align/group0_2.fasta
Wrote 500 records to genome_dir/align/group0_3.fasta
Wrote 500 records to genome_dir/align/group0_4.fasta
Wrote 500 records to genome_dir/align/group0_5.fasta
Wrote 500 records to genome_dir/align/group0_6.fasta
Wrote 406 records to genome_dir/align/group0_7.fasta


# Blast

In [26]:
group_list = []
for name in glob.glob(os.path.join(align_dir, 'group*')):
    group_list += [name]

In [27]:
group_list

['genome_dir/align/group0_1.fasta',
 'genome_dir/align/group0_6.fasta',
 'genome_dir/align/group0_4.fasta',
 'genome_dir/align/group0_5.fasta',
 'genome_dir/align/group0_2.fasta',
 'genome_dir/align/group0_7.fasta',
 'genome_dir/align/group0_3.fasta']

In [28]:
n_list = len(group_list)

In [29]:
n_list

7

In [30]:
import fnmatch, shutil

import subprocess
ref_path = args.reference_fname

for i, name in enumerate(group_list):   
    out_name = os.path.join(align_dir, f'alignment_{i}.xml')
    command_list = ['blastn', '-query', ref_path, '-subject', f'{name}', '-outfmt', '5', '-out', f'{out_name}']
    proc = subprocess.Popen(command_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.communicate()